<h1>Aprendizaje automático automatizado usando AutoKeras para predecir accidentes cerebrovasculares</h1>

<hr>

<h3>En este Notebook se va a preparar y ejecutar aprendizaje automático automatizado sobre el <i>dataset</i> 'healthcare-dataset-stroke-data.csv' (https://www.kaggle.com/fedesoriano/stroke-prediction-dataset) para conseguir un modelo capaz de predecir si un paciente con determinadas características médicas sufrirá una isquemia cerebral o si por el contrario no es propenso a ello</h3>

<hr>

<h2>1. Importación de librerías y preparación del <i>dataset</i></h2>

<p>En primer lugar, se importan los paquetes y librerías necesarias para la ejecución del código:</p>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import autokeras as ak
import kerastuner as kt

<p>A continuación, se procede a la lectura de los datos del archivo <i>healthcare-dataset-stroke-data.csv</i>. Para ello, se han visualizado los datos en bruto previamente para conocer qué atributos hay y sus respectivos tipos. Una vez resvisados, se procede a crear el <i>dataframe</i> de Pandas con los tipos de dato correctos y con el contenido del fichero CSV.</p>

In [2]:
strokes = pd.read_csv('datasets/healthcare-dataset-stroke-data.csv', dtype={'gender': 'category', 'age': float, 'hypertension': 'int8', 'heart_disease': 'int8', 'ever_married': 'category', 'work_type': 'category', 'Residence_type': 'category','avg_glucose_level': float, 'bmi': float, 'smoking_status': 'category', 'stroke': 'int8'})
strokes

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


<p>Una vez importados los datos, se procede a comprobar si se han leído correctamente mediante el siguiente código. Además, Pandas mostrará información sobre el número de filas no nulas de cada atributo. Para que el conjunto de datos sea válido para entrenar un modelo, no deberá tener ningún valor nulo.</p>

In [3]:
strokes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 5110 non-null   int64   
 1   gender             5110 non-null   category
 2   age                5110 non-null   float64 
 3   hypertension       5110 non-null   int8    
 4   heart_disease      5110 non-null   int8    
 5   ever_married       5110 non-null   category
 6   work_type          5110 non-null   category
 7   Residence_type     5110 non-null   category
 8   avg_glucose_level  5110 non-null   float64 
 9   bmi                4909 non-null   float64 
 10  smoking_status     5110 non-null   category
 11  stroke             5110 non-null   int8    
dtypes: category(5), float64(3), int64(1), int8(3)
memory usage: 200.5 KB


<p>Como se puede ver, el <i>dataset</i> tiene un total de 5110 entradas y 12 columnas. Todos los atributos tienen valor en la totalidad de las instancias, excepto la columna 9 (bmi), a la que le faltan 201 valores. Existen diferentes técnicas para solucionar este problema, tales como eliminar las filas que contengan los valores nulos, sustituirlos por un número concreto o rellenarlos con la media global de dicha propiedad. Para este caso, se ha considerado esta última solución como la más óptima. Para ello, será tan sencillo como ejecutar el siguiente código:</p>

In [4]:
strokes = strokes.fillna(strokes.mean())

<p>Comprobamos que se hayan rellenado correctamente los valores nulos:</p>

In [5]:
strokes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 5110 non-null   int64   
 1   gender             5110 non-null   category
 2   age                5110 non-null   float64 
 3   hypertension       5110 non-null   int8    
 4   heart_disease      5110 non-null   int8    
 5   ever_married       5110 non-null   category
 6   work_type          5110 non-null   category
 7   Residence_type     5110 non-null   category
 8   avg_glucose_level  5110 non-null   float64 
 9   bmi                5110 non-null   float64 
 10  smoking_status     5110 non-null   category
 11  stroke             5110 non-null   int8    
dtypes: category(5), float64(3), int64(1), int8(3)
memory usage: 200.5 KB


<p>Además de no contener valores nulos, el <i>dataset</i> deberá ser tratado para convertir los valores de los atributos categóricos en números enteros, de forma que los algoritmos de aprendizaje puedan trabajar con ellos. Para hacer esto, seleccionaremos las columnas de tipo categoría y a cada una le aplicaremos la función <i>cat.codes</i>.</p>

In [6]:
cat_columns = strokes.select_dtypes(['category']).columns
strokes[cat_columns] = strokes[cat_columns].apply(lambda x: x.cat.codes)

<p>Como se puede ver, los tipos 'category' han pasado a ser enteros, habiéndose asignado a cada valor de la categoría un número:</p>

In [7]:
strokes

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,1,67.0,0,1,1,2,1,228.69,36.600000,1,1
1,51676,0,61.0,0,0,1,3,0,202.21,28.893237,2,1
2,31112,1,80.0,0,1,1,2,0,105.92,32.500000,2,1
3,60182,0,49.0,0,0,1,2,1,171.23,34.400000,3,1
4,1665,0,79.0,1,0,1,3,0,174.12,24.000000,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,0,80.0,1,0,1,2,1,83.75,28.893237,2,0
5106,44873,0,81.0,0,0,1,3,1,125.20,40.000000,2,0
5107,19723,0,35.0,0,0,1,3,0,82.99,30.600000,2,0
5108,37544,1,51.0,0,0,1,2,0,166.29,25.600000,1,0


<p>Una vez preparado el <i>dataset</i>, se puede proceder a construir el <i>dataframe</i> que contenga las características de las que extraer la información y el que almacene la variable objetivo, en este caso el atributo binario 'stroke', el cual determinará si el paciente es propenso a sufrir una isquemia cerebral o no. Para ello, del <i>dataframe</i> completo se han eliminado las columnas 'id' (puesto que no aporta información) y 'stroke' (ya que es la variable objetivo) para construir el de características, y se ha incluido únicamente la columna 'stroke' para crear el <i>dataframe</i> objetivo.</p> 

In [8]:
features = strokes.drop(['stroke', 'id'], axis=1)
target = strokes['stroke']

<p>El contenido y la estructura de los <i>dataframes</i> son las siguientes:</p>

In [9]:
features

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,1,67.0,0,1,1,2,1,228.69,36.600000,1
1,0,61.0,0,0,1,3,0,202.21,28.893237,2
2,1,80.0,0,1,1,2,0,105.92,32.500000,2
3,0,49.0,0,0,1,2,1,171.23,34.400000,3
4,0,79.0,1,0,1,3,0,174.12,24.000000,2
...,...,...,...,...,...,...,...,...,...,...
5105,0,80.0,1,0,1,2,1,83.75,28.893237,2
5106,0,81.0,0,0,1,3,1,125.20,40.000000,2
5107,0,35.0,0,0,1,3,0,82.99,30.600000,2
5108,1,51.0,0,0,1,2,0,166.29,25.600000,1


In [10]:
target

0       1
1       1
2       1
3       1
4       1
       ..
5105    0
5106    0
5107    0
5108    0
5109    0
Name: stroke, Length: 5110, dtype: int8

<p>Otro aspecto muy importante a valorar del <i>dataset</i> es su gran desequilibrio entre los casos con accidente cerebrovascular y los pacientes no propensos a él. Como se puede ver, de 5110 casos, solo 249 son positivos. Esto provocará que si utilizamos como método de evaluación del aprendizaje una métrica como la precisión, un predictor trivial que siempre devuelva 0 (no propenso) tendrá una precisión de más del 95% con este <i>dataset</i>. Por tanto, se deberá utilizar otra métrica para evaluar la calidad del modelo de aprendizaje.</p>

In [11]:
target.value_counts()

0    4861
1     249
Name: stroke, dtype: int64

<p>De todas las métricas posibles para evaluar modelos, se ha escogido la ROC AUC, ya que esta valora que se predigan correctamente tanto los casos positivos como los negativos, pudiendo llegar únicamente al 0,5 de precisión si se predicen sólo los casos negativos, de modo que un predictor trivial tendría un mal rendimiento. De esta manera, se premiará al modelo si es capaz también de 'aprender' y predecir los casos positivos de isquemia, aunque estos sean la minoría.</p>

<p>Como último paso previo al comienzo del entrenamiento, se deberán dividir los <i>dataframes</i> de características y el objetivo en dos partes cada uno, de manera que se tengan datos para el aprendizaje (en este caso serán el 75% de ellos) y para la prueba de precisión (el 25%)</p>

In [12]:
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, target, train_size=0.75, test_size=0.25)

<h2>2. Declaración del clasificador de aprendizaje automático automatizado y búsqueda del mejor modelo</h2>

<p>La búsqueda del mejor modelo se hará utilizando el método StructuredDataClassifier, al cual se le pasa por parámetro:
<ul>
    <li>Las ejecuciones máximas, en este caso 20.</li>
    <li>El objetivo relacionado con la métrica elegida y cuya dirección será maximizar</li>
    <li>La métrica de puntuaje elegida (AUC)</li>
    <li>El tuner que utilizará el algoritmo. Por defecto utiliza 'greedy', el cual aborta la ejecución a los pocos minutos de iniciarse, al parecer por un error. Por ello, se utilizará otro distinto, en este caso 'bayesian'.</li>
    <li>El nombre del proyecto, en el cual se guardarán los modelos calculados</li>
</ul>

Se parará tras 3200 iteraciones de 250 <i>epochs</i> máximas o tras aproximadamente 24 horas. AutoKeras hará uso de la totalidad de núcleos del procesador de forma predeterminada, por lo que no hará falta indicarle ningún parámetro para ello.</p>

In [16]:
AUC_obj = kt.Objective('val_AUC', direction='max')
AUC_metric = tf.keras.metrics.AUC(name='AUC')
clf = ak.StructuredDataClassifier(max_trials=1, objective=AUC_obj, metrics=[AUC_metric], tuner='bayesian', project_name='autokeras_stroke')

INFO:tensorflow:Reloading Oracle from existing project ./autokeras_stroke/oracle.json
INFO:tensorflow:Reloading Tuner from ./autokeras_stroke/tuner0.json


<p>Se ejecuta el método <i>fit</i> para iniciar la búsqueda del mejor modelo clasificador. Debido a que AutoKeras no ofrece ninguna manera de finalizar el entrenamiento tras un cierto tiempo, se ha parado manualmente tras 25 horas y 34 minutos, habiéndose realizando con éxito 2.540 <i>trials</i> de un máximo de 250 <i>epochs</i> sobre el <i>dataset</i>, consiguiendo una precisión AUC del 0.7761 sobre el conjunto de entrenamiento. Para que AutoKeras construya la mejor <i>pipeline</i> y muestre el rendimiento del predictor, se ha ejecutado un entrenamiento de 1 solo <i>trial</i> sobre los datos guardados (ya que AutoKeras va almacenando en disco todo el proceso de entrenamiento). Esto es necesario ya que al parar manualmente el proceso de entrenamiento AutoKeras aborta inmediatamente, sin construir los resultados. Tras finalizar esa ronda, AutoKeras ha construido la mejor <i>pipeline</i> de todo el entrenamiento completo y ya es posible consultar los resultados. Toda la información de cada <i>trial</i> se encuentra en la carpeta del proyecto 'autokeras_stroke'.</p>

In [17]:
clf.fit(training_features, training_target, epochs = 250)

INFO:tensorflow:Oracle triggered exit
Epoch 1/250
120/120 [==============================] - 1s 1ms/step - loss: 0.4951 - AUC: 0.4550
Epoch 2/250
120/120 [==============================] - 0s 1ms/step - loss: 0.2333 - AUC: 0.6071
Epoch 3/250
120/120 [==============================] - 0s 1ms/step - loss: 0.2107 - AUC: 0.6565
Epoch 4/250
120/120 [==============================] - 0s 1ms/step - loss: 0.1962 - AUC: 0.6872
Epoch 5/250
120/120 [==============================] - 0s 1ms/step - loss: 0.1917 - AUC: 0.6992
Epoch 6/250
120/120 [==============================] - 0s 1ms/step - loss: 0.1869 - AUC: 0.7125
Epoch 7/250
120/120 [==============================] - 0s 1ms/step - loss: 0.1839 - AUC: 0.7209
Epoch 8/250
120/120 [==============================] - 0s 1ms/step - loss: 0.1817 - AUC: 0.7264
Epoch 9/250
120/120 [==============================] - 0s 1ms/step - loss: 0.1807 - AUC: 0.7225
Epoch 10/250
120/120 [==============================] - 0s 1ms/step - loss: 0.1832 - AUC: 0.7129
E

120/120 [==============================] - 0s 1ms/step - loss: 0.1833 - AUC: 0.6623
Epoch 169/250
120/120 [==============================] - 0s 1ms/step - loss: 0.1812 - AUC: 0.6690
Epoch 170/250
120/120 [==============================] - 0s 1ms/step - loss: 0.1830 - AUC: 0.6535
Epoch 171/250
120/120 [==============================] - 0s 1ms/step - loss: 0.1801 - AUC: 0.6737
Epoch 172/250
120/120 [==============================] - 0s 1ms/step - loss: 0.1805 - AUC: 0.6801
Epoch 173/250
120/120 [==============================] - 0s 1ms/step - loss: 0.1829 - AUC: 0.6728
Epoch 174/250
120/120 [==============================] - 0s 1ms/step - loss: 0.1828 - AUC: 0.6736
Epoch 175/250
120/120 [==============================] - 0s 1ms/step - loss: 0.1829 - AUC: 0.6719
Epoch 176/250
120/120 [==============================] - 0s 1ms/step - loss: 0.1800 - AUC: 0.6814
Epoch 177/250
120/120 [==============================] - 0s 1ms/step - loss: 0.1836 - AUC: 0.6672
Epoch 178/250
120/120 [===========

<p>Una vez finalizada la búsqueda, se muestra la precisión del mejor modelo siguiendo la métrica AUC:</p>

In [18]:
print(clf.evaluate(testing_features, testing_target))

40/40 [==============================] - 0s 1ms/step - loss: 0.1849 - AUC: 0.7123
[0.18486236035823822, 0.7123103141784668]


<p>Y se exporta a un archivo externo:</p>

In [19]:
clf.export_model()